In [1]:
import serial

ser = serial.Serial('COM3', 115200, timeout=1)
packets = []

while True:
    # Wait for start marker
    if ser.read(1) != b'\xFE':
        continue

    # Read 2-byte length
    len_low = ser.read(1)
    len_high = ser.read(1)
    if not (len_low and len_high):
        continue

    length = len_low[0] | (len_high[0] << 8)

    # Read frame
    frame = ser.read(length)

    if len(frame) == length:
        packets.append(frame)
        print(f"Captured frame ({length} bytes)")
    else:
        print("Frame error: length mismatch")


Captured frame (254 bytes)
Captured frame (611 bytes)
Captured frame (3058 bytes)
Captured frame (94 bytes)
Captured frame (3583 bytes)
Captured frame (254 bytes)
Captured frame (767 bytes)
Captured frame (114 bytes)
Captured frame (2384 bytes)
Captured frame (3819 bytes)
Captured frame (106 bytes)
Captured frame (94 bytes)
Captured frame (1494 bytes)
Captured frame (1005 bytes)
Captured frame (254 bytes)
Captured frame (94 bytes)
Captured frame (3328 bytes)
Captured frame (95 bytes)
Captured frame (94 bytes)
Captured frame (1958 bytes)
Captured frame (3898 bytes)
Captured frame (94 bytes)
Captured frame (94 bytes)
Captured frame (95 bytes)
Captured frame (94 bytes)
Captured frame (4082 bytes)
Captured frame (3307 bytes)
Captured frame (94 bytes)
Captured frame (1930 bytes)
Captured frame (862 bytes)
Captured frame (114 bytes)
Captured frame (94 bytes)
Captured frame (159 bytes)
Captured frame (106 bytes)
Captured frame (94 bytes)
Captured frame (167 bytes)
Captured frame (254 bytes)
C

KeyboardInterrupt: 

In [ ]:
import struct

target_ip = "192.168.43.81"
target_port = 80

http_get_requests = []

for frame in packets:
    # Skip Wi-Fi header (common 36 bytes)
    if len(frame) < 54:  # Minimum IP+TCP+payload
        continue
    ip_header = frame[24:44]   # 20 bytes IP header (assuming no options)
    tcp_header = frame[44:64]  # 20 bytes TCP header
    payload = frame[64:]

    # Check IP version
    version_ihl = ip_header[0]
    if (version_ihl >> 4) != 4:  # IPv4 only
        continue


    dst_ip = ".".join(str(b) for b in ip_header[16:20])
    if dst_ip != target_ip:
        continue
    print('found !! ')

    
    dst_port = (tcp_header[2] << 8) | tcp_header[3]
    if dst_port != target_port:
        continue
    # Check if payload starts with GET
    if payload.startswith(b'GET '):
        http_get_requests.append(payload.decode(errors='ignore'))
        print("Found HTTP GET:", payload.decode(errors='ignore'))


In [7]:
import re

target_ip = b"192.168.43.81"
target_port = 80

http_get_requests = []

GET_REGEX = re.compile(
    rb"GET\s+([^\r\n]+)\s+HTTP/1\.[01]\r\n",
    re.DOTALL
)

for idx, frame in enumerate(packets):

    # ---------- 1. FAST CHECK: does plaintext GET exist at all? ----------
    if b"GET " not in frame:
        continue   # encrypted or not HTTP

    print(f"[+] Raw GET-like bytes found in frame {idx}")

    # ---------- 2. Try regex extraction ----------
    matches = GET_REGEX.findall(frame)
    for m in matches:
        try:
            text = frame.decode(errors="ignore")
            http_get_requests.append(text)
            print("==== HTTP GET FOUND ====")
            print(text[:300])   # limit output
        except Exception as e:
            print("Decode error:", e)

    # ---------- 3. OPTIONAL: try IP/TCP parsing (best effort) ----------
    for i in range(len(frame) - 20):
        if frame[i] >> 4 == 4:  # IPv4 header candidate
            ihl = (frame[i] & 0x0F) * 4
            if ihl < 20:
                continue

            ip_hdr = frame[i:i+ihl]
            if len(ip_hdr) < 20:
                continue
            print('found')

            dst_ip = b".".join(str(b).encode() for b in ip_hdr[16:20])
            if dst_ip != target_ip:
                continue
            proto = ip_hdr[9]
            if proto != 6:  # TCP
                continue

            tcp_start = i + ihl
            if len(frame) < tcp_start + 20:
                continue

            tcp_hdr = frame[tcp_start:tcp_start+20]
            dst_port = (tcp_hdr[2] << 8) | tcp_hdr[3]

            if dst_port != target_port:
                continue

            payload_offset = tcp_start + ((tcp_hdr[12] >> 4) * 4)
            payload = frame[payload_offset:]

            if payload.startswith(b"GET "):
                print("==== VERIFIED HTTP GET (IP+PORT MATCH) ====")
                print(payload.decode(errors="ignore")[:300])
